In [1]:
import mwu_measures
import pandas as pd
import numpy as np
import mwu_measures.processing_corpus
from mwu_measures.corpus_helper import Fetcher

mwu_examples = pd.read_csv('MultiwordExpression_Concreteness_Ratings.csv')
mwu_examples['length'] = mwu_examples['Expression'].apply(lambda x: len(x.split()))
mwu_examples = mwu_examples.loc[(mwu_examples['length'] == 2) | (mwu_examples['length'] == 3)]
mwu_examples['Expression'] = mwu_examples['Expression'].apply(lambda x: x.lower())
this_corpus = mwu_measures.processing_corpus.process_corpus('bnc', 'small_corpus.txt', chunk_size=10000000, verbose=True)
# this_corpus = mwu_measures.processing_corpus.process_corpus(test_corpus=True)

ngram_selection = mwu_examples['Expression'].tolist()
# bigrams = [ngram.split() for ngram in ngram_selection if len(ngram.split()) == 2]
trigrams = [[' '.join([ngram.split()[0], ngram.split()[1]]), ngram.split()[2]] for ngram in ngram_selection if len(ngram.split()) == 3]
# ngram_selection = [ngram.split() for ngram in ngram_selection]
# ngram_chunks = np.array_split(ngram_selection, 100)


# bigrams = ['b d', 'c b', 'a c']


90357 lines processed
178289 lines processed
287795 lines processed
301046 lines processed
we bring the boom


In [2]:
trigrams

[['‘tis the', 'season'],
 ['8 ball', 'pool'],
 ['a bad', 'break'],
 ['a big', 'ask'],
 ['a big', 'deal'],
 ['a big', 'spender'],
 ['a bit', 'much'],
 ['a bit', 'of'],
 ['a bold', 'face'],
 ['a brave', 'face'],
 ["a busman's", 'holiday'],
 ['a cappella', 'singing'],
 ['a case', 'of'],
 ['a cash', 'cow'],
 ['a cast-iron', 'stomach'],
 ['a cat-and-dog', 'life'],
 ["a cat's", 'lick'],
 ['a checkered', 'career'],
 ['a chin', 'wag'],
 ['a cold', 'fish'],
 ['a come', 'on'],
 ['a couple', 'of'],
 ['a credit', 'to'],
 ["a curate's", 'egg'],
 ['a cut', 'above'],
 ['a degree', 'of'],
 ['a dicey', 'situation'],
 ["a dog's", 'breakfast'],
 ["a dog's", 'life'],
 ['a doubting', 'thomas'],
 ['a dry', 'run'],
 ['a fair', 'shake'],
 ['a fat', 'cat'],
 ['a fighting', 'chance'],
 ['a firm', 'grip'],
 ["a fool's", 'paradise'],
 ['a foregone', 'conclusion'],
 ['a gem', 'of'],
 ['a good', 'deal'],
 ['a good', 'few'],
 ['a good', 'many'],
 ['a good', 'sport'],
 ['a gray', 'area'],
 ['a great', 'deal'],
 ['a g

In [3]:
helper = Fetcher(this_corpus.corpus_conn)

In [5]:
helper.allocate_filtered_db('big_1', 'ug_3')

In [7]:
x = helper.get_ngram_scores('big_1', 'ug_3', 3) # assocs don't work in trigrams

In [34]:
# TODO: fix assoc with trigrams
# TODO: function to get all ngrams with different lengths
# TODO: batches to not crash memory when using complete corpus. Worth it? faster because of the simpler joins? worth it to hash?